In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
!pip install mitdeeplearning --quiet
import mitdeeplearning as mdl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
pip install midiutil


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for midiutil: filename=MIDIUtil-1.2.1-py3-none-any.whl size=54569 sha256=7b77ef8c510f63ef82fb442652158049ca2eb6d11ed1a8b2ad3ccd99f52291b3
  Stored in directory: /root/.cache/pip/wheels/af/43/4a/00b5e4f2fe5e2cd6e92b461995a3a97a2cebb30ab5783501b0
Successfully built midiutil


In [ ]:
#load the dataset
songs = mdl.lab1.load_training_data()

Found 817 songs in text


In [ ]:
# Print an example song to inspect it
example_song = songs[0]
print("\nExample song: ")
print(example_song)



Example song: 
X:1
T:Alexander's
Z: id:dc-hornpipe-1
M:C|
L:1/8
K:D Major
(3ABc|dAFA DFAd|fdcd FAdf|gfge fefd|(3efe (3dcB A2 (3ABc|!
dAFA DFAd|fdcd FAdf|gfge fefd|(3efe dc d2:|!
AG|FAdA FAdA|GBdB GBdB|Acec Acec|dfaf gecA|!
FAdA FAdA|GBdB GBdB|Aceg fefd|(3efe dc d2:|!


# Data Preparation

In [ ]:
# Join all songs into a single string for easier processing
# Limiting to 50,000 characters if necessary to reduce memory usage
songs_joined = "\n\n".join(songs[:50000])

# Character Mapping

In [ ]:
# Identify all unique characters in the dataset
vocab = sorted(set(songs_joined))
print("There are", len(vocab), "unique characters in the dataset")

# Create mappings from character to integer and integer to character
char_to_int = {char: i for i, char in enumerate(vocab)}
int_to_char = {i: char for i, char in enumerate(vocab)}

There are 83 unique characters in the dataset


#Data Reshaping and Normalization

In [ ]:
# Data preparation with a smaller sequence length
seq_length = 50  # Reduced sequence length
X, y = [], []

# Create sequences of characters from the dataset
for i in range(len(songs_joined) - seq_length):
    X.append([char_to_int[char] for char in songs_joined[i:i + seq_length]])
    y.append(char_to_int[songs_joined[i + seq_length]])
# Convert lists to numpy arrays for model compatibility
X = np.array(X)
y = np.array(y)

# Reshape X for compatibility with RNN/LSTM/GRU layers: [samples, time steps, features]
# Normalize X by dividing by the length of the vocabulary
X = np.reshape(X, (X.shape[0], X.shape[1], 1)) / float(len(vocab))

# Model Definitions

In [ ]:
# Model 1: Simple RNN with reduced units
def create_rnn_model():
    model = Sequential()
    model.add(SimpleRNN(64, input_shape=(X.shape[1], X.shape[2]), activation='relu'))
    model.add(Dense(len(vocab), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

# Model 2: LSTM with reduced units
def create_lstm_model():
    model = Sequential()
    model.add(LSTM(64, input_shape=(X.shape[1], X.shape[2]), activation='relu'))
    model.add(Dense(len(vocab), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

# Model 3: GRU with reduced units
def create_gru_model():
    model = Sequential()
    model.add(GRU(64, input_shape=(X.shape[1], X.shape[2]), activation='relu'))
    model.add(Dense(len(vocab), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

# Instantiate Models

In [ ]:
# Create and compile each model
rnn_model = create_rnn_model()
lstm_model = create_lstm_model()
gru_model = create_gru_model()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


# Display Model Summaries

In [ ]:

print("Summary for Simple RNN Model:")
rnn_model.summary()

print("\nSummary for LSTM Model:")
lstm_model.summary()

print("\nSummary for GRU Model:")
gru_model.summary()

Summary for Simple RNN Model:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 64)                  │           4,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 83)                  │           5,395 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,619 (37.57 KB)

 Trainable params: 9,619 (37.57 KB)

...

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 64)                  │          12,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 83)                  │           5,395 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,259 (71.32 KB)

 Trainable params: 18,259 (71.32 KB)

 Non-trainable params: 0 (0.00 B)

# Model Training and Music Generation
* Model Training:
The models (RNN, LSTM, GRU) are trained on the data using the fit() function with specified epochs and batch size.
* Music Generation:
The generate_music() function is used for generating music using the trained models by applying Top-K sampling to predict the next characters in the sequence.

In [ ]:

# Train the models with fewer epochs and a batch size
rnn_model.fit(X, y, epochs=50, batch_size=64, verbose=1)
lstm_model.fit(X, y, epochs=50, batch_size=64, verbose=1)
gru_model.fit(X, y, epochs=50, batch_size=64, verbose=1)

# Function to generate music using Top-K sampling
def generate_music(model, seed, num_chars=500, top_k=5):
    generated_sequence = seed
    for _ in range(num_chars):
        input_seq = np.reshape([char_to_int[char] for char in generated_sequence[-seq_length:]], (1, seq_length, 1))
        input_seq = input_seq / float(len(vocab))  # Normalize the input

        pred = model.predict(input_seq, verbose=0).squeeze()

        # Apply Top-K sampling
        top_k_indices = np.argsort(pred)[-top_k:]
        top_k_pred = pred[top_k_indices]
        top_k_pred = top_k_pred / np.sum(top_k_pred)  # Normalize

        next_char_index = np.random.choice(top_k_indices, p=top_k_pred)
        next_char = int_to_char[next_char_index]
        generated_sequence += next_char
    return generated_sequence

# Generate music using each model
seed_text = songs_joined[:seq_length]  # Using the first `seq_length` characters as seed
rnn_generated = generate_music(rnn_model, seed=seed_text, num_chars=500)
lstm_generated = generate_music(lstm_model, seed=seed_text, num_chars=500)
gru_generated = generate_music(gru_model, seed=seed_text, num_chars=500)

# Print the generated music
print("\nGenerated music with RNN:")
print(rnn_generated[:500])  # Print first 500 characters

print("\nGenerated music with LSTM:")
print(lstm_generated[:500])  # Print first 500 characters

print("\nGenerated music with GRU:")
print(gru_generated[:500])  # Print first 500 characters


There are 83 unique characters in the dataset
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3135/3135 ━━━━━━━━━━━━━━━━━━━━ 44s 13ms/step - accuracy: 0.1800 - loss: 3.0915
Epoch 2/50
3135/3135 ━━━━━━━━━━━━━━━━━━━━ 41s 13ms/step - accuracy: 0.3521 - loss: 2.2404
Epoch 3/50
3135/3135 ━━━━━━━━━━━━━━━━━━━━ 82s 13ms/step - accuracy: 0.3641 - loss: 2.1560
Epoch 4/50
3135/3135 ━━━━━━━━━━━━━━━━━━━━ 41s 13ms/step - accuracy: 0.3866 - loss: 2.0240
Epoch 5/50
3135/3135 ━━━━━━━━━━━━━━━━━━━━ 82s 13ms/step - accuracy: 0.4034 - loss: 1.9452
Epoch 6/50
3135/3135 ━━━━━━━━━━━━━━━━━━━━ 82s 13ms/step - accuracy: 0.4139 - loss: 1.8858
Epoch 7/50
3135/3135 ━━━━━━━━━━━━━━━━━━━━ 82s 13ms/step - accuracy: 0.4056 - loss: 1.9329
Epoch 8/50
3135/3135 ━━━━━━━━━━━━━━━━━━━━ 42s 13ms/step - accuracy: 0.4263 - loss: 1.8167
Epoch 9/50
3135/3135 ━━━━━━━━━━━━━━━━━━━━ 81s 13ms/step - accuracy: 0.4144 - loss: 1.8925
Epoch 10/50
3135/3135 ━━━━━━━━━━━━━━━━━━━━ 42s 13ms/step - accuracy: 0.4255 - loss: 1.8247
Epoch 11/50
3135/3135 ━━━━━━━━━━━━━━━━━━━━ 82s 13ms/step - accuracy: 0.4353 - loss: 1.7674
Epoch 12/50
3135/31

#MIDI File Generation from generated notes Text

In [ ]:
# Function to generate and save a MIDI file from ABC text
def save_to_midi_from_abc(abc_text, file_name="generated_music.mid", tempo=120):
    # Mapping ABC notes to MIDI note values (central octave)
    note_to_midi = {
        'C': 60, 'D': 62, 'E': 64, 'F': 65, 'G': 67, 'A': 69, 'B': 71,
        'c': 72, 'd': 74, 'e': 76, 'f': 77, 'g': 79, 'a': 81, 'b': 83
    }
    # Extract notes from ABC text using regex
    notes = re.findall(r'[A-Ga-g][#]?', abc_text)

    # Convert notes to MIDI values
    midi_notes = [note_to_midi[note] for note in notes if note in note_to_midi]

    midi = MIDIFile(1)  # Create a single-track MIDI file
    track = 0
    time = 0  # Start time in seconds

    midi.addTrackName(track, time, "Generated Track")
    midi.addTempo(track, time, tempo)

    # Set parameters for each note
    channel = 0
    volume = 100  # Note volume
    duration = 0.5  # Note duration in seconds

    # Add each note to the MIDI file
    for pitch in midi_notes:
        midi.addNote(track, channel, pitch, time, duration, volume)
        time += duration

    # Save MIDI file
    with open(file_name, "wb") as output_file:
        midi.writeFile(output_file)
    print(f"MIDI file saved as {file_name}")

In [ ]:
generated_rnn_text="""
X:1
T:Alexander's
Z: id:dc-hornpipe-1
M:C|
L:1/8
K:|
M:1/de
d: e26/4 K:1/8
K:F Maroa
og|egdc affg|dged dfed|ffge eaee|faed dage|feff afge|fagg adfd|gedg gefd|afff afaf|fadd ggaf|edfd eeegagfeadae dagf|gedf gfdg|addg aegfngggeffae aaefggfd|dge gafdaaa afadgddf aff edfc|gfde fadg|dfd gdfdeaf fgadagd geeangfg gddgafe fffdafge|gge aefddad efffadf eggfedfg|aBBA F2d|feed gffd|aeeg fffd|fedf feec|fafe dfff|gff gfdfggededdg gfaeeefd|afdd eaef|aea afdaefdgadd efgeagagaaff efae ggfcdgf daefffafeaea gafa|a

"""

generated_lstm_text="""
X:1
T:Alexander's
Z: id:dc-hornpipe-1
M:C|
L:1/8
K:A Major dc|eBG2 ceef|dddd gdgg|dBGA G2A|A2GG EDGA|!
AABF F2G|dfdf e2B2|BGA2 BAF2|ABAG GGA|]!

X:13
T:Frlla oe Crols
Z: id:dc-reed-198
M:C
L:1/8
K:A Major
AGAD DAEF|B3GD EGBF|A3F BFEF|BGBA B2B|!
A2G2 DFAB|ABFG EFBF|BABA ABBF|EAEE FEG2|]!

X:362
T:Prdliell
s: id:dc-reel-108
M:C
L:1/8
K:A Mixolydian
dd|BAFA c2dAdA|e2dd eeed|eddd cAGc|]edA G2:|!

X:162
T:Kusny Conn
Z: id:dc-reel-104
M:C
L:1/8
K:G Dorian
e|dBde fdeedddec deBA|BFcec e2dec|f2
gfag aegf
"""

generated_gru_text="""
X:1
T:Alexander's
Z: id:dc-hornpipe-1
M:C|
L:1/8
K:A Mixolydian
D|ddff dAGA|G2FB GAF2|B2AG AFEB|G2FE AFFE:|!
gddf dgdg|e2dd adeg|eddg edBd|egea e2eg|!
g2fg adeg|ffga efge|gefde fddd|cfeec efdB|!

X:31
T:Bal oe tha Rhleeo'sfrhe
Z: id:dc-reel-20
M:C
L:1/8
K:E Mixolydian
A,|F2BG A2DG|A3EG BGGA|DGAF B2BA|E2FG DFDD|!
G2AE ADFE|A2AA BAEA|G2dd geBG|A2AA DEED|!
D3AG BFFE|B2dd gdBd|d2Ad dBBF|GFFE GED2:|!
defe edBA|c2af fded|d3Bc BGAB|d2ef edB2|]!
fdfa a2ef|d3de ffBd|f2ef dded|f2ef d2:2|!

X:338
T:Wail oh
"""

# Call save_to_midi_from_abc for each generated text and save them as separate MIDI files
save_to_midi_from_abc(generated_rnn_text, file_name="generated_music_rnn.mid")
save_to_midi_from_abc(generated_lstm_text, file_name="generated_music_lstm.mid")
save_to_midi_from_abc(generated_gru_text, file_name="generated_music_gru.mid")

MIDI file saved as generated_music_rnn.mid
MIDI file saved as generated_music_lstm.mid
MIDI file saved as generated_music_gru.mid
